In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.optimize import curve_fit
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
all_training_data = pd.DataFrame()


def logistic_function(t, beta_0, beta_1, beta_2):
    return beta_2 / (1 + np.exp(-(beta_0 + (t * beta_1))))



for i in range(50):  # Assuming there are 50 training data files
    file_name = f'/Users/brasov/Downloads/training_data/degradation_data/item_{i}.csv'
    training_data = pd.read_csv(file_name)
    all_training_data = pd.concat([all_training_data, training_data], ignore_index=True)


def fit_logistic_model(all_training_data):
    # Normalize data
    scaler = StandardScaler()
    time_scaled = scaler.fit_transform(all_training_data[['time (months)']].values.reshape(-1, 1)).flatten()
    crack_length = all_training_data['crack length (arbitary unit)'].values

    # More informed initial guess based on your data analysis
    p0 = [1, 0.1, max(crack_length)]  # Example: [initial beta_0, initial beta_1, initial beta_2]

    # Parameter bounds
    bounds = ([0, -np.inf, 0], [np.inf, np.inf, np.inf])  # Example bounds

    # Fit the logistic model with bounds and method
    popt, pcov = curve_fit(logistic_function, time_scaled, crack_length, p0=p0, bounds=bounds, method='trf')
    print(popt)
    return popt  # This ensures popt is returned as an array-like object of parameter





def simulate_crack_growth_with_noise(time, popt, sigma_process_0=0.01, sigma_process_1=0.001, sigma_process_2=0.01,
                                     sigma_observation=0.05):

    # Process noise
    beta_0_noisy = 0.8854633 + np.random.normal(0, sigma_process_0)
    beta_1_noisy = 0.18762 + np.random.normal(0, sigma_process_1)
    beta_2_noisy = 0.95 + np.random.normal(0, sigma_process_2)

    crack_lengths_no_noise = logistic_function(time, beta_0_noisy, beta_1_noisy, beta_2_noisy)

    crack_lengths_observed = crack_lengths_no_noise + np.random.normal(0, sigma_observation, len(time))

    return crack_lengths_observed


popt = fit_logistic_model(all_training_data);


# Particle Filter Functions
def initialize_particles(num_particles, parameter_bounds):
    return np.random.uniform(low=parameter_bounds['low'], high=parameter_bounds['high'], size=(num_particles, 3))


def propagate_particles(particles, process_noise_std):
    return particles + np.random.normal(0, process_noise_std, particles.shape)


def update_weights(particles, observation, time, observation_noise_std):
    predicted = np.array([logistic_function(time, *particle) for particle in particles])
    likelihoods = np.exp(-0.5 * ((observation - predicted) / observation_noise_std) ** 2)
    weights = likelihoods / np.sum(likelihoods)
    return weights


def resample_particles(particles, weights):
    N = len(particles)
    positions = (np.arange(N) + np.random.random()) / N
    indexes = np.zeros(N, 'i')
    cumulative_sum = np.cumsum(weights)
    i, j = 0, 0
    while i < N:
        if positions[i] < cumulative_sum[j]:
            indexes[i] = j
            i += 1
        else:
            j += 1
    return particles[indexes], np.ones_like(weights) / N



def particle_filter_integration(all_training_data, num_particles=10000,
                                parameter_bounds={'low': [0, 0, 0], 'high': [10, 1, 10]}, sigma_process_0=0.01,
                                sigma_process_1=0.001, sigma_process_2=0.01, observation_noise_std=0.05):
    particles = initialize_particles(num_particles, parameter_bounds)
    weights = np.ones(num_particles) / num_particles

    for index, row in all_training_data.iterrows():
        time = row['time (months)']
        observation = row['crack length (arbitary unit)']

        # Apply process noise to particles
        particles[:, 0] += np.random.normal(0, sigma_process_0, num_particles)
        particles[:, 1] += np.random.normal(0, sigma_process_1, num_particles)
        particles[:, 2] += np.random.normal(0, sigma_process_2, num_particles)

        # Update weights based on observation
        weights = update_weights(particles, observation, time, observation_noise_std)
        particles, weights = resample_particles(particles, weights)

    estimated_parameters = np.average(particles, axis=0, weights=weights)
    return estimated_parameters


# Predict if the crack length will exceed the threshold within the next 6 months
def crack_growth_failure(beta_0, beta_1, beta_2, y_th=0.85, months=6):
    future_months = np.arange(1, months + 1)
    crack_lengths = logistic_function(future_months, beta_0, beta_1, beta_2)
    return np.any(crack_lengths >= y_th)


# Load failure data
failure_data = pd.read_csv('/Users/brasov/Downloads/failure_data.csv')

# Load training data for each item ID
training_data_path = "/Users/brasov/Downloads/training_data/degradation_data/"


predictions = []
for item_id in failure_data['item_id']:
    file_path = os.path.join(training_data_path, f"item_{item_id}.csv")
    training_data = pd.read_csv(file_path)

    # Determine if the RUL indicates failure within 6 months
    prediction = int(training_data['rul (months)'].iloc[-1] < 6)

    predictions.append({'item_id': f'item_{item_id}', 'label': prediction})

# Convert predictions to DataFrame
predictions_df = pd.DataFrame(predictions)
predictions = []

estimated_parameters = particle_filter_integration(all_training_data)

# Iterate through testing data files for prediction
testing_data_dir = '/Users/Brasov/Downloads/testing_data/'
for i in range(50):
    file_path = os.path.join(testing_data_dir, f'testing_item_{i}.csv')
    testing_data = pd.read_csv(file_path)

    max_time = testing_data['time (months)'].max()
    max_crack = testing_data['crack length (arbitary unit)'].max()

    future_months = np.arange(1, 7) + max_time

    # Determine if any future crack lengths exceed the failure threshold
    future_crack_lengths_noisy = simulate_crack_growth_with_noise(future_months, *estimated_parameters)
    failure_prediction = int(np.any(future_crack_lengths_noisy >= 0.85))

    # Append prediction for this item
    predictions.append({'item_index': f'item_{i}', 'label': failure_prediction})

# Convert predictions to DataFrame and save to CSV
predictions_df = pd.DataFrame(predictions)
print(predictions_df)
predictions_df.to_csv('/Users/Brasov/Downloads/submission100.csv', index=False)
# Save predictions for 50 robots to a CSV file

# Load the submission file
submission_df = pd.read_csv('/Users/Brasov/Downloads/submission100.csv')

# Count the number of times 1 appears in the 'label' column
num_ones = submission_df['label'].sum()

print(f"Number of times 1 appears in the 'label' column: {num_ones}")